# Planejamento

## Etapas
- [x] ~~Tratar tipos dos dados~~
- [ ] Tratar dados vazios
- [X] Tratar formatação de números para comparações
- [ ] Montar análises de Pareto
    - [X] Acumulado de prêmio ganho
    - [X] Acumulado de número de apólices
    - Quebrar por ramo
    - Quebrar por produto
    - Quebrar por localização
- [ ] Montar análises cohort de duração de vigência

# Próximos passos
- [ ] Pareto de motivos de cancelamento

# Preparação dos dados

In [2]:
# Importação de pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# configurações de visualização
pd.set_option('display.max_columns', None)

## Dados de Apólices

In [ ]:
# Importação dos dados de apólices
df_apolice = pd.read_excel('Apolices_Emitidas_.xlsx')

# Ajustar nome de colunas
df_apolice_colunas_nomes ={
	"nrProposta":						"Proposta",
	"nrApolice":						"Apólice",
	"nrEndosso":						"Endosso",
	"nrEndossoAssociado":				"Endosso Associado",
	"dsEndosso":						"Descrição do Endosso",
	"statusApolice":					"Status da Apólice",
	"idGrupoRamoSeguro":				"ID do Grupo do Ramo",
	"RamoSeguro":						"Número do Ramo",
	"nmGrupoRamoSeguro":				"Nome do Grupo do Ramo",
	"dtEmissaoMovimento":				"Data de Emissão",
	"dtIniVigencia":					"Data de Início da Vigência",
	"dtFimVigencia":					"Data de Fim da Vigência",
	"vlPremioComercialSeguroMoeda":		"Valor do Prêmio Comercial",
	"vlCustoAquisicaoOperacionalMoeda":	"Valor do Custo de Aquisição",
	"vlIOF":							"Valor do IOF",
	"vlAdicionalFracionamentoMoeda":	"Valor do Adicional de Fracionamento",
	"vlIS":								"Valor de IS",
	"nrTotalFracionamento":				"Número Total de Fracionamentos",
	"nrCpfCnpjCorretorLider":			"CPF/CNPJ do Corretor Líder",
	"nmRazaoSocialCorretorLider":		"Razão Social do Corretor Líder",
	"dtNnascimento":					"Data de Nascimento",
	"nmSexo":							"Sexo",
	"tipoPessoa":						"Tipo de Pessoa",
	"cdProdutoApolice":					"Código do Produto",
	"nmProduto":						"Nome do Produto",
	"nmCep":							"CEP",
	"nmCidade":							"Cidade",
	"nmUF":								"UF"
}
df_apolice = df_apolice.rename(columns=df_apolice_colunas_nomes)
memory_usage_before_df_apolice = df_apolice.memory_usage(deep=True)
types_before = df_apolice.dtypes

# Ajustar tipos de colunas
df_apolice_colunas_tipos ={
	"Descrição do Endosso":					"category",
	"Status da Apólice":					"category",
	"ID do Grupo do Ramo":					"category",
	"Número do Ramo":						"category",
	"Nome do Grupo do Ramo":				"category",
	"Data de Emissão":						"datetime64[ns]",
	"Data de Início da Vigência":			"datetime64[ns]",
	"Data de Fim da Vigência":				"datetime64[ns]",
	"Valor do Prêmio Comercial":			"float64",
	"Valor do Custo de Aquisição":			"float64",
	"Valor do IOF":							"float64",
	"Valor do Adicional de Fracionamento":	"float64",
	"Valor de IS":							"float64",
	"Número Total de Fracionamentos":		"category",
	"CPF/CNPJ do Corretor Líder":			"category",
	"Razão Social do Corretor Líder":		"category",
	"Data de Nascimento":					"datetime64[ns]",
	"Sexo":									"category",
	"Tipo de Pessoa":						"category",
	"Código do Produto":					"category",
	"Nome do Produto":						"category",
	"CEP":									"string",
	"Cidade":								"category",
	"UF":									"category"
	}
df_apolice = df_apolice.astype(df_apolice_colunas_tipos)
memory_usage_after_df_apolice = df_apolice.memory_usage(deep=True)
types_after = df_apolice.dtypes

# Gerar parquet
df_apolice.to_parquet("apolices.parquet", index=False)

In [ ]:
# Comparar uso de memória de apólices
memory_usage_df_apolice = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_apolice,
    'Memory Usage After': memory_usage_after_df_apolice,
    'Percentage Difference': (memory_usage_after_df_apolice
                              - memory_usage_before_df_apolice)
                              / memory_usage_before_df_apolice * 100
})
memory_usage_df_apolice.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_apolice['Memory Usage Before'].sum(),
    memory_usage_df_apolice['Memory Usage After'].sum(),
    (memory_usage_df_apolice['Memory Usage After'].sum() 
     - memory_usage_df_apolice['Memory Usage Before'].sum()) 
     / memory_usage_df_apolice['Memory Usage Before'].sum() * 100
]

memory_usage_df_apolice

In [103]:
def plot_pareto_contagem(df: pd.DataFrame, agrupa_por, campo_agrupado, tipo_agg: str = None) -> pd.DataFrame:
    """
    Gera um gráfico de Pareto a partir de um DataFrame agrupado por uma coluna e contando outra.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        agrupa_por (str): Nome da coluna a ser usada como agrupamento.
        campo_agrupado (str): Nome da coluna cujos valores serão agregados.
        tipo_agg (str, opcional): Tipo de agregação (ex: 'count', 'nunique', 'sum').
                                  Se não informado, será inferido com base no tipo do campo_agrupado.

    Returns:
        pd.DataFrame: DataFrame com totais por categoria e colunas de porcentagens.
    """
    # Inferir agregação caso não seja fornecida
    if tipo_agg is None:
        tipo = df[campo_agrupado].dtype
        if pd.api.types.is_numeric_dtype(tipo):
            tipo_agg = 'sum'
        else:
            tipo_agg = 'nunique'

    # Agrupamento dinâmico
    df_acumulado = df.groupby(agrupa_por, observed=True).agg({
        campo_agrupado: tipo_agg
    }).reset_index()

    df_acumulado.sort_values(by=campo_agrupado, ascending=False, inplace=True)
    df_acumulado['per_do_todo'] = df_acumulado[campo_agrupado] / df_acumulado[campo_agrupado].sum() * 100
    df_acumulado['per_accum'] = df_acumulado['per_do_todo'].cumsum()
    df_acumulado['% do todo'] = df_acumulado['per_do_todo'].apply(lambda x: f"{x:.2f}%")
    df_acumulado['% acumulada'] = df_acumulado['per_accum'].apply(lambda x: f"{x:.2f}%")

    # Cálculo de Pareto 80/20
    pareto1 = df_acumulado.loc[df_acumulado['per_accum'] > 80].iloc[0]['per_accum']
    pareto2 = len(df_acumulado.loc[df_acumulado['per_accum'] <= pareto1])
    pareto3 = len(df_acumulado)
    print(f"{(pareto2 / pareto3) * 100:.2f}% das categorias de '{agrupa_por}' correspondem a {pareto1:.2f}% do total de '{campo_agrupado}'")

    # Gráfico de Pareto
    fig, ax1 = plt.subplots(figsize=(12, 6))
    bars = ax1.bar(df_acumulado[agrupa_por].astype(str), df_acumulado[campo_agrupado], color='skyblue')
    ax1.set_xlabel(agrupa_por)
    ax1.set_ylabel(f'{tipo_agg} de {campo_agrupado}', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    ax2 = ax1.twinx()
    ax2.plot(df_acumulado[agrupa_por].astype(str), df_acumulado['per_accum'], color='red', marker='o')
    ax2.set_ylabel('% Acumulada', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    ax2.set_ylim(0, 110)

    plt.title(f'Gráfico de Pareto - {campo_agrupado} por {agrupa_por}')
    ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

    for i, val in enumerate(df_acumulado['per_accum']):
        ax2.annotate(f'{val:.2f}%', (df_acumulado[agrupa_por].astype(str).iloc[i], val),
                     textcoords="offset points", xytext=(0, 5), ha='center',
                     fontsize=8, color='red')

    plt.tight_layout()
    plt.show()

    return df_acumulado.drop(columns=['per_do_todo', 'per_accum'])

In [ ]:
plot_pareto_contagem(df_apolice, agrupa_por='Número do Ramo', campo_agrupado='Valor do Prêmio Comercial', tipo_agg='sum')

In [84]:
df_apolice.to_parquet("apolices.parquet", index=False)

In [9]:
df_apolice_parquet = pd.read_parquet("apolices.parquet")
df_apolice_parquet

,Proposta,Apólice,Endosso,Endosso Associado,Descrição do Endosso,Status da Apólice,ID do Grupo do Ramo,Número do Ramo,Nome do Grupo do Ramo,Data de Emissão,Data de Início da Vigência,Data de Fim da Vigência,Valor do Prêmio Comercial,Valor do Custo de Aquisição,Valor do IOF,Valor do Adicional de Fracionamento,Valor de IS,Número Total de Fracionamentos,CPF/CNPJ do Corretor Líder,Razão Social do Corretor Líder,Data de Nascimento,Sexo,Tipo de Pessoa,Código do Produto,Nome do Produto,CEP,Cidade,UF
0,205775,2023011905310165685,000001T,0.0,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-08-06,2024-07-07,2024-07-08,0.00,0.00,0.00,0.0,51870.00,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1958-12-06,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,85550-000,CORONEL VIVIDA,PR
1,206438,2023011905310166200,000001T,0.0,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-08-22,2024-07-12,2024-07-13,0.00,0.00,0.00,0.0,56513.00,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1961-01-03,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,85440-000,UBIRATÃ,PR
2,207799,2023011905310167362,000001R,0.0,ENDOSSO DE RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-07-01,2024-07-01,2024-07-31,63.22,6.32,0.00,0.0,43516.00,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1982-03-23,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,87470-000,MARILUZ,PR
3,208027,2023011905310167598,000001R,0.0,ENDOSSO DE RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-07-09,2024-07-09,2024-07-29,27.14,2.71,0.00,0.0,68098.00,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1962-09-27,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,85904-000,TOLEDO,PR
4,208276,2023011905310167777,000002R,NaN,ENDOSSO DE RESTITUIÇÃO DE PREMIO,EXPIRADO,5,31,AUTOMÓVEL - CASCO,2024-07-02,2024-06-10,2024-07-28,148.77,14.87,7.22,0.0,93163.00,2,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1971-06-14,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,87060-668,MARINGÁ,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172450,98226570,1009811000080,49,0.0,COBRANÇA,"EXPIRADA ULTIMO ENDOSSO, PORÉM ENDOSSO 0 VIGENTE",11,98,RURAL,2025-07-10,2025-05-31,2025-06-30,2666.30,1066.52,0.00,0.0,1271836.25,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,NaT,NaN,PJ,413,SANCOR SEGUROS VIDA RURAL TM VINCULADO,86010390,LONDRINA,PR
172451,98226573,1009811000069,61,0.0,COBRANÇA,"EXPIRADA ULTIMO ENDOSSO, PORÉM ENDOSSO 0 VIGENTE",11,98,RURAL,2025-07-11,2025-05-31,2025-06-30,54.35,21.74,0.00,0.0,70945.40,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,NaT,NaN,PJ,413,SANCOR SEGUROS VIDA RURAL TM VINCULADO,85601260,FRANCISCO BELTRÃO,PR
172452,9836,1008113002532,11,0.0,COBRANÇA,EXPIRADA ENDOSSO 0 E ÚLTIMO ENDOSSO,13,81,PESSOAS INDIVIDUAL,2024-10-28,2024-12-09,2025-12-09,216.46,64.92,0.82,0.0,1052190.96,12,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1964-03-01,MASCULINO,PF,64,SANCOR SEGUROS AP PREMIADO,85900040,TOLEDO,PR
172453,99,1011401000006,0,NaN,APÓLICE,VIGENTE,1,14,PATRIMONIAL,2025-01-27,2025-01-24,2026-01-24,0.00,0.00,0.00,0.0,NaN,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1966-05-04,MASCULINO,PF,17,RESIDENCIAL FÁCIL,86067970,LONDRINA,PR


## Dados de Sinistros

In [86]:
# Importação dos dados de sinistros
df_sinistro = pd.read_excel('Apolices_Sinistradas_Sicoob_.xlsx')

# Ajustar nome de colunas
df_sinistro_colunas_nomes ={
	"nrApolice":							"nr_apolice",
	"nrEndosso":							"nr_endosso",
	"dsCoberturaSeguro":					"ds_cobertura",
	"CdCoberturaSeguro":					"cd_cobertura",
	"idGrupoRamoSeguro":					"id_grp_ramo",
	"idRamoSeguro":							"id_ramo",
	"nrSinistro":							"nr_sinistro",
	"MovimentoSinistro":					"nm_movto_sinistro",
	"dtEmissaoEventoSinistro":				"dt_emissao_mvto_sinistro",
	"vlEventoSinistroMoeda":				"vl_evento_sinsitro_moeda",
	"vlEventoSinistroBRL":					"vl_evento_sinsitro_brl",
	"dtOcorrenciaSinistro":					"dt_ocorrencia_sinistro",
	"dtAvisoSinistro":						"dt_aviso_sinistro",
	"dtRegistroSinistro":					"dt_registro_sinistro",
	"cdUFOcorrenciaSinistro":				"uf_ocorrencia_sinistro",
	"nrCpfCnpjContraparte":					"cpf_cnpj_contraparte",
	"nmRazaoSocialContraparte":				"razao_social_contraparte",
	"cdProcessoJudicial":					"cd_processo_judicial",
}
df_sinistro = df_sinistro.rename(columns=df_sinistro_colunas_nomes)
memory_usage_before_df_sinistro = df_sinistro.memory_usage(deep=True)
types_before = df_sinistro.dtypes

# Ajustar tipos de colunas
df_sinistro_colunas_tipos ={
	"ds_cobertura":							"category",
	"cd_cobertura":							"category",
	"id_grp_ramo":							"category",
	"id_ramo":								"category",
	"nm_movto_sinistro":					"category",
	"dt_emissao_mvto_sinistro":				"datetime64[ns]",
	"vl_evento_sinsitro_moeda":				"float64",
	"vl_evento_sinsitro_brl":				"float64",
	"dt_ocorrencia_sinistro":				"datetime64[ns]",
	"dt_aviso_sinistro":					"datetime64[ns]",
	"dt_registro_sinistro":					"datetime64[ns]",
	"uf_ocorrencia_sinistro":				"category",
	"cpf_cnpj_contraparte":					"float64",
}
df_sinistro_tipo = df_sinistro.astype(df_sinistro_colunas_tipos)
memory_usage_after_df_sinistro = df_sinistro_tipo.memory_usage(deep=True)
types_after = df_sinistro.dtypes

# Exportar para parquet
df_sinistro_tipo.to_parquet("sinistros.parquet", index=False)

In [ ]:
# Comparar uso de memória de sinistro
memory_usage_df_sinistro = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_sinistro,
    'Memory Usage After': memory_usage_after_df_sinistro,
    'Percentage Difference': (memory_usage_after_df_sinistro
                              - memory_usage_before_df_sinistro)
                              / memory_usage_before_df_sinistro * 100
})
memory_usage_df_sinistro.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_sinistro['Memory Usage Before'].sum(),
    memory_usage_df_sinistro['Memory Usage After'].sum(),
    (memory_usage_df_sinistro['Memory Usage After'].sum() 
     - memory_usage_df_sinistro['Memory Usage Before'].sum()) 
     / memory_usage_df_sinistro['Memory Usage Before'].sum() * 100
]

memory_usage_df_sinistro